In [3]:
import numpy as np
import pandas as pd

import time
from tqdm import tqdm

import glob
import re

In [2]:
file_list = glob.glob('/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/*.csv')

In [3]:
file_list

['/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/터키 BIST 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/Dow Jones Shanghai 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/스위스 SMI 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/닛케이 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/호주 S&P_ASX 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/DAX 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/사우디아라비아 Tadawul 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/RTSI 지수 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/다우존스 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/영국 FTSE 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/Nifty 50 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/S&P 500 내역.csv',
 '/Users/hwanghyeonjun/

In [4]:
file_names = []
for path in tqdm(file_list):
	file_name = path.split('/')[-1]
	file_names.append(file_name)

100%|██████████| 43/43 [00:00<00:00, 227721.05it/s]


In [5]:
file_names

['터키 BIST 내역.csv',
 'Dow Jones Shanghai 내역.csv',
 '스위스 SMI 내역.csv',
 '닛케이 내역.csv',
 '호주 S&P_ASX 내역.csv',
 'DAX 내역.csv',
 '사우디아라비아 Tadawul 내역.csv',
 'RTSI 지수 내역.csv',
 '다우존스 내역.csv',
 '영국 FTSE 내역.csv',
 'Nifty 50 내역.csv',
 'S&P 500 내역.csv',
 '스페인 IBEX 내역.csv',
 '대만 가권 내역.csv',
 '상하이종합 내역.csv',
 '네덜란드 AEX 내역.csv',
 'VN 30 내역.csv',
 'S&P_BMV IPC 내역.csv',
 '벨기에 BEL 내역.csv',
 'FTSE China A50 내역.csv',
 '코스닥 내역.csv',
 'BSE 인도 센섹스 30 내역.csv',
 '코스피 50 내역.csv',
 '헝가리 BUX 내역.csv',
 '러셀 2000 지수 내역.csv',
 '항셍 내역.csv',
 '폴란드 WIG 20 내역.csv',
 '스웨덴 OMXS 내역.csv',
 '코스피지수 내역.csv',
 '브라질 보베스파 내역.csv',
 'SZSE Component 내역.csv',
 '_캐나다 S&P_TSX 내역.csv',
 '나스닥종합지수 내역.csv',
 'TA 35 내역.csv',
 '유로 스톡스 50 내역.csv',
 'SET Index 내역.csv',
 'ATX 내역.csv',
 '프랑스 CAC 내역.csv',
 'PSI 내역.csv',
 '러시아 MOEX Russia 내역.csv',
 'CBOE Volatility Index 내역.csv',
 '인도네시아 IDX 내역.csv',
 '이탈리아 FTSE MIB 내역.csv']

In [6]:
test_df = pd.read_csv('/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/헝가리 BUX 내역.csv')

In [7]:
test_df.dtypes

날짜       object
종가       object
오픈       object
고가       object
저가       object
거래량     float64
변동 %     object
dtype: object

In [8]:
print(test_df['거래량'].dtypes)

float64


In [9]:
for a in test_df['거래량']:
	if str(a) == 'nan':
		b = str(a).replace('nan', '0')
		print(b)
		# c = a.replace(b, '0')
		# print(c)
	elif list(a)[-1] == 'K':
		b = list(a)[-1]
		c = a.replace(b, '')
		c = float(c) * 1000
		print(c)
	elif list(a)[-1] == 'M':
		b = list(a)[-1]
		c = a.replace(b, '')
		c = round(float(c) * 1000000)
		print(c)
	elif list(a)[-1] == 'B':
		b = list(a)[-1]
		c = a.replace(b, '')
		c = round(float(c) * 1000000000)
		print(c)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


AttributeError: 'float' object has no attribute 'find'

In [308]:
# # 데이터 형식 바꾸기
# def data_format_change(file_list: list, file_names: list):
# 	for idx, file in enumerate(tqdm(file_list)):
# 		date_regx = '\D?\s?'
# 		persent_regx = '\d+[%]$'
#
# 		prices = ['종가', '오픈', '고가', '저가']
#
# 		volumes = []
#
# 		load_df = pd.read_csv(file)
# 		df = load_df.astype(str)
#
# 		df.rename(columns={'변동 %': '변동'}, inplace=True)
#
# 		df['날짜'] = df['날짜'].apply(lambda x: re.sub(date_regx, '', x))
#
# 		for price in prices:
# 			df[price] = df[price].apply(lambda x: x.replace(',', '')).astype('float')
#
# 		df['거래량'] = df['거래량'].apply(lambda x: x.replace(',', ''))
#
# 		for volume in df['거래량'].values:
# 			cash_unit = list(volume)[-1]
#
# 			if str(volume) == 'nan':
# 				n = str(volume).replace('nan', '0')
# 				volumes.append(n)
# 			elif cash_unit == 'K':
# 				k = volume.replace(cash_unit, '')
# 				k_result = float(k) * 1000
# 				volumes.append(k_result)
# 			elif cash_unit == 'M':
# 				m = volume.replace(cash_unit, '')
# 				m_result = round(float(m) * 1000000)
# 				volumes.append(m_result)
# 			elif cash_unit == 'B':
# 				b = volume.replace(cash_unit, '')
# 				b_result = round(float(b) * 1000000000)
# 				volumes.append(b_result)
#
# 		df['거래량'] = volumes
#
# 		if not bool(re.match(persent_regx, list(df['변동'])[0])):
# 			df['변동'] = df['변동'].apply(lambda x: x.replace('%', '')).astype('float')
#
# 		df.sort_values(['날짜'], inplace=True)

In [309]:
data_format_change(file_list)

100%|██████████| 43/43 [00:00<00:00, 124.78it/s]


In [10]:
def data_format_change(file_list: list):
	df_list = []
	for idx, file in enumerate(tqdm(file_list)):
		date_regx = '\D?\s?'
		persent_regx = '\d+[%]$'

		prices = ['종가', '오픈', '고가', '저가']

		volumes = []

		load_df = pd.read_csv(file)
		df = load_df.astype(str)

		df.rename(columns={'변동 %': '변동'}, inplace=True)

		df['날짜'] = df['날짜'].apply(lambda x: re.sub(date_regx, '', x))

		for price in prices:
			df[price] = df[price].apply(lambda x: x.replace(',', '')).astype('float')

		df['거래량'] = df['거래량'].apply(lambda x: x.replace(',', ''))

		for volume in df['거래량'].values:
			cash_unit = list(volume)[-1]

			if str(volume) == 'nan':
				n = str(volume).replace('nan', '0')
				volumes.append(n)
			elif cash_unit == 'K':
				k = volume.replace(cash_unit, '')
				k_result = float(k) * 1000
				volumes.append(k_result)
			elif cash_unit == 'M':
				m = volume.replace(cash_unit, '')
				m_result = round(float(m) * 1000000)
				volumes.append(m_result)
			elif cash_unit == 'B':
				b = volume.replace(cash_unit, '')
				b_result = round(float(b) * 1000000000)
				volumes.append(b_result)

		df['거래량'] = volumes

		if not bool(re.match(persent_regx, list(df['변동'])[0])):
			df['변동'] = df['변동'].apply(lambda x: x.replace('%', '')).astype('float')

		df.sort_values(['날짜'], inplace=True)

		df_list.append(df)

	return df_list


In [11]:
df_list = data_format_change(file_list)

100%|██████████| 43/43 [00:00<00:00, 191.62it/s]


In [12]:
def df_save(df_list: list, df_names: list, save_path: str, file_extension='csv'):
	for idx, dataframe in enumerate(tqdm(df_list)):
		try:
			dataframe.to_csv(save_path + df_names[idx] + '.' + file_extension)
		except:
			print('저장 오류')

	return print('파일 저장됨!')

In [317]:
df_save(df_list, df_names, )

NameError: name 'df_list' is not defined

In [140]:
rate_file_list = glob.glob(
	'/Users/hwanghyeould not convert string to fonjun/Documents/GitHub/data/selenium/환율/*.csv'
	)
bonds_file_list = glob.glob('/Users/hwanghyeonjun/Documents/GitHub/data/selenium/세계지수/*.csv')
commodities_file_list = glob.glob('/Users/hwanghyeonjun/Documents/GitHub/data/selenium/원자재/*.csv')
coins_file_list = glob.glob('/Users/hwanghyeonjun/Documents/GitHub/data/selenium/암호화폐/*.csv')

In [43]:
# # 날짜 순서로 오름차순정렬
#
# def dataframe_srot(file_list):
# 	file_names = []
#
# 	for path in tqdm(file_list):
# 		file_name = path.split('/')[-1]
# 		file_names.append(file_name)
#
# 	for idx, file in tqdm(enumerate(file_list)):
# 		df = pd.read_csv(file)
# 		df.sort_values(['날짜'], inplace=True)
# 		df['날짜'] = df['날짜'].apply(lambda x: x.replace(' ', ''))
# 		df.to_csv('/Users/hwanghyeonjun/Documents/GitHub/data/selenium/' + file_names[idx], index=False)

In [47]:
# dataframe_srot(rate_file_list)

100%|██████████| 23/23 [00:00<00:00, 450789.68it/s]
23it [00:00, 270.72it/s]


In [10]:
data = pd.read_csv('/Users/hwanghyeonjun/Documents/GitHub/data/selenium/채권/독일_1년_채권_수익율_내역.csv')

In [11]:
data

,날짜,종가,오픈,고가,저가,변동 %
0,2020-01-01,-0.656,-0.668,-0.656,-0.668,3.31%
1,2020-01-02,-0.606,-0.628,-0.569,-0.653,-7.62%
2,2020-01-03,-0.612,-0.585,-0.584,-0.641,0.99%
3,2020-01-06,-0.604,-0.593,-0.587,-0.626,-1.31%
4,2020-01-07,-0.613,-0.594,-0.580,-0.621,1.49%
...,...,...,...,...,...,...
741,2022-11-07,2.269,2.259,2.348,2.220,1.48%
742,2022-11-08,2.270,2.282,2.344,2.256,0.04%
743,2022-11-09,2.241,2.282,2.364,2.228,-1.28%
744,2022-11-10,2.236,2.250,2.304,2.175,-0.22%


In [13]:
data.insert(5,'거래량','0',allow_duplicates=False)

In [25]:
data.drop('거래량', axis=1, inplace=True)

In [26]:
if '거래량' in list(data.columns):
	print('있음')
else:
	print('없음')

없음
